In [2]:
import pandas as pd
import os
import glob
import numpy as np

# setting notebook
pd.set_option('display.max_columns', None)

In [3]:
path = "../../data/raw/00-po-all"

all_files = glob.glob(os.path.join(path, "*.csv"))

li = []

for filename in all_files:
    df = pd.read_csv(filename, dtype=str ,index_col=None, header=10)
    li.append(df)

po_cbn= pd.concat(li, axis=0, ignore_index=True)

In [4]:
#Drop null rows
po_nonull = po_cbn.dropna(subset=['Số đơn hàng\n(Order No)']).reset_index(drop=True)

In [5]:
#Rename columns
po_rename = po_nonull.rename(columns={'Unnamed: 24': 'Chứng từ KH ký nhận', 'Unnamed: 26': 'Chứng từ khác'}) 

In [6]:
#Convert datetime
date_cols = ['Ngày về\n(Receiving Date)',
              'Hạn chót giao hàng\n(Delivery Deadline)',
              'Ngày thực tế giao hàng']
po_rename[date_cols] = po_rename[date_cols].apply(pd.to_datetime, format='%d/%m/%Y')

po_rename['Chứng từ KH ký nhận'] = np.where(~po_rename['Chứng từ KH ký nhận'].astype(str).str.contains(r'^\d{2}\/\d{2}\/\d{4}', regex=True, na=False), pd.NaT, po_rename['Chứng từ KH ký nhận'])
po_rename['Chứng từ KH ký nhận'] = po_rename['Chứng từ KH ký nhận'].apply(pd.to_datetime, format='%d/%m/%Y')



In [7]:
po_joined = po_rename.copy()

In [8]:
po_joined.columns.tolist()

['Số\nTT',
 'Mã phiếu đề xuất\n(Code)',
 'Số đơn hàng\n(Order No)',
 'Mã khách hàng\n(Customer Code)',
 'Tên khách hàng\n(Customer Name)',
 'Mã chủ hàng',
 'Tên chủ hàng',
 " Trị giá đơn hàng\n(PO's value) ",
 'Ngày về\n(Receiving Date)',
 'Hạn chót giao hàng\n(Delivery Deadline)',
 'Ngày giao hàng theo yêu cầu KH\n(Delivery date requested by customer)',
 'Ngày lên Phiếu xuất kho',
 'Ngày bàn giao đơn hàng cho Giao Nhận',
 'Ngày thực tế giao hàng',
 'Đơn vị vận chuyển',
 'Ngày thực tế giao hàng đến Điểm giao',
 'Người giao hàng (Shipper delivery by)',
 'Lý do hủy đơn hàng (Reason of cancel order)',
 'Tình trạng hóa đơn\n(Invoice status)',
 'Unnamed: 19',
 'Unnamed: 20',
 'Unnamed: 21',
 'Chứng từ trả về',
 'Unnamed: 23',
 'Chứng từ KH ký nhận',
 'Unnamed: 25',
 'Chứng từ khác',
 'Người nhặt hàng',
 'Tình trạng đơn hàng',
 'Tình trnạg đơn hàng Smartlog',
 ' Số ngày xử lý ',
 'Loại đơn hàng',
 'Ngày dự kiến giao hàng',
 'Mã Phiếu đề xuất của HP/ RW',
 'Kho xuất',
 'Loại phiếu',
 'Mã NV 0